In [3]:
import tensorflow as tf
import pandas as pd

In [2]:
model = tf.keras.models.load_model('../model/twitter-spam')
model.summary()

2021-10-28 08:25:37.245007: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:29:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-10-28 08:25:37.320170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:29:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-10-28 08:25:37.320584: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:29:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-10-28 08:25:37.321509: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 71, 16)            649152    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_24 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 129       
Total params: 670,145
Trainable params: 670,145
Non-trainable params: 0
_________________________________________________________________


In [36]:
squid_tweets = pd.read_json('../dataset/squid-tweets/squid_tweets.json').rename(columns={0: 'tweets'})

In [37]:
squid_tweets['tweets']

0       #BTS, #TXT, And "#SquidGame" Nominated For 202...
1       tim pool watched squid game and somehow missed...
2              Squid Game is short for Squidward’s Gambit
3       Having tons of money doesn’t mean anything if ...
4       Squid game\n\nBAZINGA MV BUKAS NA\n@SB19Offici...
                              ...                        
1019                   Squid game https://t.co/8M8al6iWgb
1020       I gave squid game a shot and it’s pretty good.
1021    OMG this last ass scene from Squid Game is dra...
1022    @allkpopBuzz @allkpop At least Lebron watched ...
1023                      WHERE IS MY SQUID GAME COSTUME?
Name: tweets, Length: 1024, dtype: object

In [39]:
# Load tokenizer
import pickle
with open('../dataset/basic-tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

squid_seq = tokenizer.texts_to_sequences(squid_tweets['tweets'])
squid_pad = tf.keras.preprocessing.sequence.pad_sequences(squid_seq, padding='post', maxlen=71)

In [41]:
squid_pad

array([[ 1367, 36679,    12, ...,     0,     0,     0],
       [ 2336,  5279,  1570, ...,     0,     0,     0],
       [13813,   253,    17, ...,     0,     0,     0],
       ...,
       [  591,    19,   153, ...,     0,     0,     0],
       [    1,  3744,    23, ...,     0,     0,     0],
       [  182,    17,    25, ...,     0,     0,     0]], dtype=int32)

In [43]:
squid_tweets['is-spam'] = model.predict(squid_pad)

In [45]:
squid_tweets['is-spam'] = [1 if i >= 0.5 else 0 for i in squid_tweets['is-spam']]
squid_tweets.tail()

,tweets,is-spam
1019,Squid game https://t.co/8M8al6iWgb,1
1020,I gave squid game a shot and it’s pretty good.,0
1021,OMG this last ass scene from Squid Game is dra...,0
1022,@allkpopBuzz @allkpop At least Lebron watched ...,0
1023,WHERE IS MY SQUID GAME COSTUME?,0


In [46]:
squid_tweets.to_excel('../dataset/squid-tweets/squid-dataset.xlsx')